# State Design Pattern

In [1]:
class IState{
    public:
    virtual ~IState() {}
    virtual void insertQuarter() = 0;
    virtual void ejectQuarter() = 0;
    virtual void turnCrank() = 0;
    virtual void dispense() = 0;
};

In [2]:
#include <iostream>
//class CGumballMachine;

class CSoldState: public IState
{
public:
CSoldState(CGumballMachine& gumballMachine):m_GumballMachine(gumballMachine){}
virtual ~CSoldState() override {}
virtual void insertQuarter() override final {std::cout << "Please wait, we are already giving you a gumball\n";}
virtual void ejectQuarter() override final{std::cout << "You already turned the crank\n";}
virtual void turnCrank() override final{std::cout << "Turning twice does't get another ball\n";}
virtual void dispense() override final{
    m_GumballMachine.releaseBall();
    if (m_GumballMachine.getCount() > 0)
    {
        m_GumballMachine.setState(m_GumballMachine.getNoQuarterState());
    }
    else
    {
        std::cout << "Sold out...\n";
        m_GumballMachine.setState(m_GumballMachine.getSoldOutState());
    }
}
private:
CGumballMachine& m_GumballMachine;
    
};

input_line_9:12:21: error: member access into incomplete type 'CGumballMachine'
    m_GumballMachine.releaseBall();
                    ^
input_line_9:2:8: note: forward declaration of 'CGumballMachine'
 class CGumballMachine;
       ^
input_line_9:13:9: error: use of undeclared identifier 'gumballMachine'
    if (gumballMachine.getCount() > 0)
        ^
input_line_9:15:25: error: member access into incomplete type 'CGumballMachine'
        m_GumballMachine.setState(m_GumballMachine.getNoQuarterState());
                        ^
input_line_9:2:8: note: forward declaration of 'CGumballMachine'
 class CGumballMachine;
       ^
input_line_9:20:25: error: member access into incomplete type 'CGumballMachine'
        m_GumballMachine.setState(m_GumballMachine.getSoldOutState());
                        ^
input_line_9:2:8: note: forward declaration of 'CGumballMachine'
 class CGumballMachine;
       ^


Interpreter Error: 

In [ ]:
class CSoldOutState: public IState
{
public:
CSoldOutState(CGumballMachine& gumballMachine):m_GumballMachine(gumballMachine){}
virtual ~CSoldOutState() override {}
virtual void insertQuarter() override final {}
virtual void ejectQuarter() override final{}
virtual void turnCrank() override final{}
virtual void dispense() override final{}

private:
CGumballMachine& m_GumballMachine;
    
};

In [ ]:
class CNoQuarterState: public IState
{
public:
CNoQuarterState(CGumballMachine& gumballMachine):m_GumballMachine(gumballMachine){}
virtual ~CNoQuarterState() override {}
virtual void insertQuarter() override final 
{
    std::cout << "You inserted a quarter\n";
    m_GumballMachine.setState(getHasQuarterState());
    
}
virtual void ejectQuarter() override final
{
 std::cout << "You havent inserted a quarter\n";
}
virtual void turnCrank() override final
{
 std::cout << "You turned but there is no quarter\n";
}
virtual void dispense() override final
{
 std::cout << "You need to pay first\n";
}
    
private:
CGumballMachine& m_GumballMachine;
    
};

In [ ]:
//class CGumballMachine;
class CHasQuarterState: public IState
{
public:
CHasQuarterState(CGumballMachine& gumballMachine):m_GumballMachine(gumballMachine){}
virtual ~CHasQuarterState() override {}
virtual void insertQuarter() override final {std::cout << "You can't insert another quarter\n";}
virtual void ejectQuarter() override final
{
    std::cout << "Quarter returned\n";
    m_GumballMachine.setState(getNoQuarterState());
}
virtual void turnCrank() override final
{
    std::cout << "You turned...\n";
    m_GumballMachine.setState(getSoldState());
}
virtual void dispense() override final{std::cout << "No gumball dispensed\n";}
    
private:
CGumballMachine& m_GumballMachine;    
};

In [ ]:
class CWinnerState: public IState
{
public:
CWinnerState(CGumballMachine& gumballMachine):m_GumballMachine(gumballMachine){}
virtual ~CWinnerState() override {}
virtual void insertQuarter() override final {std::cout << "Error:insertQuarter in winner state\n";}
virtual void ejectQuarter() override final{std::cout << "Error ejectQuarter in winner state\n";}
virtual void turnCrank() override final{std::cout << "Error turnCrank in winner state\n";}
virtual void dispense() override final
{
    std::cout << "You are a winner\n";
    m_GumballMachine.releaseBall();
    if (m_GumballMachine.getCount() == 0)
    {
        m_GumballMachine.setState(m_GumballMachine.getSoldOutState());
    }
    else
    {
        m_GumballMachine.releaseBall();
        if (m_GumballMachine.getCount() > 0)
        {
            m_GumballMachine.setState(m_GumballMachine.getNoQuarterState());
        }
        else
        {
            std::cout << "out of ball...\n";
            m_GumballMachine.setState(m_GumballMachine.getSoldOutState());
        }
    }    
}
private:
CGumballMachine& m_GumballMachine;    
};

In [ ]:
class CGumballMachine
{
    public:
    CGumballMachine(int numberGumballs):m_state(nullptr),m_count(numberGumballs)
    {
        soldOutState = new CSoldOutState(this);
        noQuarterState = new CNoQuarterState(this);
        hasQuarterState = new CHasQuarterState(this);
        soldState = new CSoldState(this);
        m_state = soldOutState;
        if (m_count > 0)
        {
            m_state = noQuarterState;
        }
    }
    virtual ~CGumballMachine(){}
    
    void insertQuarter()
    {
        m_state->insertQuarter();
    }
    
    void ejectQuarter()
    {
        m_state->ejectQuarter();
    }
    
    void turnCrank()
    {
        m_state->turnCrank();
        m_state->dispense();
    }
    
    void setState(IState* state)
    {
        m_state = state;
    }
    
    void releaseBall()
    {
        std::cout << "A gumball comes rolling out the slot...";
        if (m_count !=0)
        {
            m_count = m_count - 1;
        }
    }
    
    IState* getSoldOutState(){return soldOutState;}
    IState* getNoQuarterState() {return noQuarterState;}
    IState* getHasQuarterState() {return hasQuarterState;}
    IState* getSoldState() {return soldState;}
    
    private:
    IState* soldOutState;
    IState* noQuarterState;
    IState* hasQuarterState;
    IState* soldState;
    
    IState* m_state;
    int m_count = 0;
}